In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

# Read and label data from KenPom

In [2]:
# Base url, and a lambda func to return url for a given year
base_url = 'http://kenpom.com/index.php'
url_year = lambda x: '%s?y=%s' % (base_url, str(x) if x != 2020 else base_url)

# Years on kenpom's site (could also scrape this and 
# set as a list if you want to be more dynamic)
years = range(2002, 2020)

In [3]:
# Create a method that parses a given year and spits out a raw dataframe
def import_raw_year(year):
    """
    Imports raw data from a ken pom year into a dataframe
    """
    f = requests.get(url_year(year))
    soup = BeautifulSoup(f.text)
    table_html = soup.find_all('table', {'id': 'ratings-table'})

    # Weird issue w/ <thead> in the html
    # Prevents us from just using pd.read_html
    # Let's find all the thead contents and just replace/remove them
    # This allows us to easily put the table row data into a dataframe using panda
    thead = table_html[0].find_all('thead')

    table = table_html[0]
    for x in thead:
        table = str(table).replace(str(x), '')

#    table = "<table id='ratings-table'>%s</table>" % table
    df = pd.read_html(table)[0]
    df['year'] = year
    return df

In [4]:
# Import all the years into a singular dataframe
df = None
for x in years:
    df = pd.concat( (df, import_raw_year(x)), axis=0) \
        if df is not None else import_raw_year(2002)

C:\Users\Ray\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Ray\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [5]:
# Column rename based off of original website
df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'AdjEM', 
             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
             'SOS AdjEM', 'SOS AdjEM Rank', 'SOS OppO', 'SOS OppO Rank',
             'SOS OppD', 'SOS OppD Rank', 'NCSOS AdjEM', 'NCSOS AdjEM Rank', 'Year']

In [6]:
# Lambda that returns true if given string is a number and a valid seed number (1-16)
valid_seed = lambda x: True if str(x).replace(' ', '').isdigit() \
                and int(x) > 0 and int(x) <= 16 else False

# Use lambda to parse out seed/team
df['Seed'] = df['Team'].apply(lambda x: x[-2:].replace(' ', '') \
                              if valid_seed(x[-2:]) else np.nan )

df['Team'] = df['Team'].apply(lambda x: x[:-2] if valid_seed(x[-2:]) else x)

In [7]:
# Split W-L column into wins and losses
df['Wins'] = df['W-L'].apply(lambda x: int(re.sub('-.*', '', x)) )
df['Losses'] = df['W-L'].apply(lambda x: int(re.sub('.*-', '', x)) )
df.drop('W-L', inplace=True, axis=1)

In [8]:
# Reorder columns just cause I'm OCD
df=df[[ 'Year', 'Rank', 'Team', 'Conference', 'Wins', 'Losses', 'Seed','AdjEM', 
             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
             'SOS AdjEM', 'SOS AdjEM Rank', 'SOS OppO', 'SOS OppO Rank',
             'SOS OppD', 'SOS OppD Rank', 'NCSOS AdjEM', 'NCSOS AdjEM Rank']]

# Match Team IDs to entries

In [9]:
teams_df = pd.read_csv('Stage2DataFiles/TeamSpellings.csv', sep='\,', engine='python')

In [10]:
teams_df.head()

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101


In [12]:
df['Team'] = df['Team'].str.lower().str.strip()
df = pd.merge(df, teams_df, left_on=['Team'], right_on = ['TeamNameSpelling'], how='left')
df = df.drop(['TeamNameSpelling'], axis=1)

In [13]:
df.head()

,Year,Rank,Team,Conference,Wins,Losses,Seed,AdjEM,AdjustO,AdjustO Rank,...,Luck Rank,SOS AdjEM,SOS AdjEM Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS AdjEM,NCSOS AdjEM Rank,TeamID
0,2002,1,duke,ACC,31,4,1,34.19,121.0,1,...,223,9.87,18,109.1,13,99.2,31,6.66,34,1181.0
1,2002,2,cincinnati,CUSA,31,4,1,30.19,118.1,7,...,165,6.58,57,106.3,66,99.7,42,3.48,80,1153.0
2,2002,3,maryland,ACC,32,4,1,29.25,119.2,4,...,104,9.88,16,109.1,11,99.3,33,1.62,120,1268.0
3,2002,4,kansas,B12,33,4,1,28.99,118.7,5,...,109,10.66,10,110.3,4,99.6,40,8.32,23,1242.0
4,2002,5,oklahoma,B12,31,5,2,26.04,114.9,20,...,69,8.77,26,109.0,15,100.2,62,-0.45,169,1328.0


In [14]:
df.to_csv('kenpom.csv', index=False)